# Code for Demo - When we downlaod the new video from you tube you can use this code for the prediction. 

In [ ]:
import csv
import numpy as np
import random
import glob
import os.path
import sys
import operator
import threading
from processor import process_image
from keras.utils import to_categorical

In [2]:
sequence_path = os.path.join('data', 'youtubevideo_seq')

In [3]:
seq_length = 40
max_frames = 300  # max number of frames a video can have for us to use it

# for lstm and mlp 

data_type = 'features'
features_length=2048
input_shape = (seq_length, features_length)




In [4]:
##################################              get data 

with open(os.path.join('data', 'youtubevideo.csv'), 'r') as fin:
    reader = csv.reader(fin)
    data = list(reader)

with open('classes.csv', 'r') as fin2:
    reader = csv.reader(fin2)
    classes = list(reader)

    

In [8]:
def split_train_test():

    train = []
    test = []
    finaltest=[]
    for item in data:
        if item[0] == 'train':
            train.append(item)
        if item[0] == 'finaltest':
            finaltest.append(item)
        if item[0] == 'youtubevideo':
            test.append(item)
    return train, test, finaltest

In [10]:
def build_image_sequence(frames):

    return [process_image(x, image_shape) for x in frames]




In [11]:

def get_extracted_sequence( data_type, sample):

    filename = sample[1]
    path = os.path.join(sequence_path, filename + '-' + str(seq_length) + '-' + data_type + '.npy')
    if os.path.isfile(path):
        return np.load(path)
    else:
        return None




In [12]:
def get_frames_for_sample(sample):

    path = os.path.join('data', sample[0])
    filename = sample[1]
    images = sorted(glob.glob(os.path.join(path, filename + '*jpg')))
    return images


In [13]:

def get_filename_from_image(filename):
    parts = filename.split(os.path.sep)
    return parts[-1].replace('.jpg', '')

In [14]:

def rescale_list(input_list, size):

    assert len(input_list) >= size

    # Get the number to skip between iterations.
    skip = len(input_list) // size

    # Build our new output.
    output = [input_list[i] for i in range(0, len(input_list), skip)]

    # Cut off the last one if needed.
    return output[:size]




In [15]:
def get_all_sequences_in_memory(train_test_finaltest, data_type):

    # Get the right dataset.
    train, test, finaltest = split_train_test()

    if train_test_finaltest == 'youtubevideo':
        data= test
    if train_test_finaltest == 'train':
        data = train
    if train_test_finaltest == 'finaltest':
        data= finaltest




    print("Loading %d samples into memory for %sing." % (len(data), train_test_finaltest))
#     print(data)

    X, y = [], []
    for row in data:
#         print("****************************************************")

#         print(row)
        
        if data_type == 'images':
            print(row)
            frames = get_frames_for_sample(row)
            frames = rescale_list(frames, seq_length)
            print(frames)

            # Build the image sequence
            sequence = build_image_sequence(frames)

        else:
            sequence = get_extracted_sequence(data_type, row)

            if sequence is None:
                print("Can't find sequence. Did you generate them?")
                raise

        X.append(sequence)


    return np.array(X)



In [16]:
from keras.preprocessing import image
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.models import Model, load_model
from keras.layers import Input
import numpy as np
from tqdm import tqdm

In [17]:

# Get model with pretrained weights.
base_model = InceptionV3(weights='imagenet', include_top=True)



In [18]:
# We'll extract features at the final pool layer.
inception_model = Model(inputs=base_model.input,outputs=base_model.get_layer('avg_pool').output)

In [19]:
pbar = tqdm(total=len(data))
for video in data:

    # Get the path to the sequence for this video.
    path = os.path.join('data', 'youtubevideo_seq', video[1] + '-' + str(seq_length) + '-features')  # numpy will auto-append .npy

    # Check if we already have it.
    if os.path.isfile(path + '.npy'):
        pbar.update(1)
        continue

    # Get the frames for this video.
    frames = get_frames_for_sample(video)

    # Now downsample to just the ones we need.
    frames = rescale_list(frames, seq_length)


    # Now loop through and extract features to build the sequence.
    sequence = []
    for image_path in frames:
        print(image_path)
        img = image.load_img(image_path, target_size=(299, 299))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)

        # Get the prediction.
        features = inception_model.predict(x)

        features = features[0]
        
        sequence.append(features)

    # Save the sequence.
    np.save(path, sequence)

    pbar.update(1)

pbar.close()


100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 499.02it/s]


In [20]:
from keras.layers import Dense, Flatten, Dropout, ZeroPadding3D
from keras.layers.recurrent import LSTM
from keras.models import Sequential, load_model
from keras.optimizers import Adam, RMSprop
from keras.layers.wrappers import TimeDistributed
from keras.layers.convolutional import (Conv2D, MaxPooling3D, Conv3D,
    MaxPooling2D)
from collections import deque
import sys
from keras.models import model_from_json
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, CSVLogger
import time


In [21]:
X_lstm_test =get_all_sequences_in_memory('youtubevideo', data_type)

Loading 1 samples into memory for youtubevideoing.


In [22]:
nb_classes= len(classes)
features_length=2048
input_shape = (seq_length, features_length)

In [23]:
json_file = open('model_lstm.json', 'r')
loaded_model_json = json_file.read()
json_file.close()


In [24]:
loaded_model_lstm = model_from_json(loaded_model_json)
# load weights into new model
loaded_model_lstm.load_weights("model_lstm.h5")
print("Loaded model from disk")

Loaded model from disk


In [25]:
metrics = ['accuracy']
if nb_classes >= 10:
    metrics.append('top_k_categorical_accuracy')
    
optimizer = Adam(lr=1e-5, decay=1e-6)
loaded_model_lstm.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=metrics)

In [ ]:
loaded_model_lstm_predict = loaded_model_lstm.predict(X_lstm_test)

In [45]:
num_predictions = 20
for predict_index, predicted_y in enumerate(loaded_model_lstm_predict):
#     actual_label = classes[np.argmax(Y_lstm_test[predict_index])]
    predicted_label = classes[np.argmax(predicted_y)]
    print('Predicted Label = %s' % (predicted_label))
    if predict_index == num_predictions:
        break

Predicted Label = ['ApplyLipstick']


In [42]:
loaded_model_lstm_predict

array([[  6.55350462e-02,   9.31042552e-01,   3.25587948e-08,
          7.16654611e-08,   2.48733323e-09,   6.67529832e-07,
          3.94910728e-07,   5.03382012e-08,   9.17524901e-08,
          3.13972492e-08,   1.54040833e-08,   4.23546638e-08,
          3.59253609e-05,   1.15917019e-05,   7.33283088e-08,
          2.98972680e-10,   2.44809950e-08,   4.26027533e-08,
          6.52211725e-08,   2.38578254e-03,   7.11141368e-08,
          1.92988399e-08,   1.32940457e-07,   7.19577019e-06,
          6.65318964e-08,   1.32089428e-09,   1.01755809e-07,
          1.41146739e-08,   2.02503813e-07,   8.67067513e-07,
          6.37725464e-07,   2.54265217e-07,   6.86103760e-07,
          5.45694027e-04,   1.17531249e-06,   5.64623917e-07,
          1.77552035e-08,   1.37604124e-08,   6.39593125e-08,
          1.15054320e-07,   2.22435460e-06,   3.19431711e-05,
          3.88111948e-06,   8.93583252e-09,   3.28522987e-08,
          3.18627440e-06,   3.27468086e-07,   3.51556722e-08,
        